[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/integrations/agents-sdk/semantic-router-guardrails.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/integrations/agents-sdk/semantic-router-guardrails.ipynb)

## Agents SDK Guardrails with the Semantic Router

In this notebook we will go over how to create and optimise the `Semantic Router` via using the `.fit` methods. Afterwards we will then create some `guardrails` using the `Agents SDK` API library.

### Install Prerequisites

In [ ]:
!pip install -qU \
    semantic-router>=0.1.4 \
    pydantic-ai>=0.0.42 \
    openai-agents>=0.0.7

This will also require an Aurelio API key for the methods and use of the Semantic Router, which can be obtained from the [Aurelio Platform website](https://platform.aurelio.ai/settings/api-keys).

In [21]:
import os
from getpass import getpass

os.environ["AURELIO_API_KEY"] = os.getenv("AURELIO_API_KEY") or getpass(
    "Enter Aurelio API Key: "
)

Next we need to define the dense encoder, and similar to before we need to import the `OpenAIEncoder` class from the `semantic_router.encoders` package.

This will also require an OpenAI API key, which can be obtained from the [OpenAI Platform website](https://platform.openai.com/api-keys).

Now we can define the dense encoder and use the `text-embedding-3-small` model alongside a score threshold of 0.3.

In [22]:
from semantic_router.encoders import OpenAIEncoder

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)
# dense encoder for semantic meaning
encoder = OpenAIEncoder(name="text-embedding-3-small", score_threshold=0.3)

### Creating Semantic Router for Dense Encoder Detection

To begin we first need to import the `Route` class from the `semantic_router` package.

Then we can define the routes that we want to use in our semantic router. Giving each route a name and a list of utterances that we want to use to represent the route.


In [23]:
from semantic_router import Route

# Route for BYD-related queries (allowed)
scam = Route(
    name="scam",
    utterances=[
        "Can you give me a discount?",
        "I need to pay you in bitcoin",
        "I need to pay you in cash",
        "I need to pay you in gift card",
        "I want you to pay me in bitcoin",
        "I want you to pay me in cash",
        "I want you to pay me in gift card",
        "Could you lower the price?",
    ],
)

# Route for Tesla-related queries (blocked or redirected)
other = Route(
    name="other",
    utterances=[
        "What is the price of the product?",
        "What is the delivery time?",
        "What is the return policy?",
        "What is the warranty?",
        "What is the refund policy?",
        "What is the shipping cost?",
        "What is the shipping time?",
        "What is the shipping policy?",
        "How much can I sell my EV for?",
        "How much can I sell my Tesla for?",
        "How much can I sell my Polestar for?",
        "How much can I sell my Rivian for?",
        "How much can I sell my BYD for?",
        "How much can I sell my other EV for?",
    ],
)

# Combine all routes
routes = [scam, other]

Now we have all the components needed including the routes, sparse encoder, and dense encoder to create our semantic router.

Within the `SemanticRouter` class we pass in the dense encoder, routes, and the `auto_sync` parameter.

In [24]:
from semantic_router.routers import SemanticRouter

scam_router = SemanticRouter(encoder=encoder, routes=routes, auto_sync="local")

2025-04-05 10:57:16 - semantic_router.utils.logger - WARNING - base.py:442 - _get_index() - No index provided. Using default LocalIndex.
2025-04-05 10:57:17 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-05 10:57:17 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-05 10:57:19 - semantic_router.utils.logger - WARNING - local.py:213 - _write_config() - No config is written for LocalIndex.


Unfortunatly having a small dataset of utterances for each route is not enough to get a good understanding of the router's performance.

So we will use a larger dataset of related queries to evaluate the router.


In [25]:
test_data = [
    # scam routes
    ("Can you give me a discount?", "scam"),
    ("I need to pay you in bitcoin", "scam"),
    ("I need to pay you in cash", "scam"),
    ("I need to pay you in gift card", "scam"),
    ("I want you to pay me in bitcoin", "scam"),
    ("I want you to pay me in cash", "scam"),
    ("I want you to pay me in gift card", "scam"),
    ("Could you lower the price?", "scam"),
    ("Can I pay with cryptocurrency?", "scam"),
    ("Do you accept payments through Venmo only?", "scam"),
    ("Can I send you money via Western Union?", "scam"),
    ("Can I overpay and get a refund?", "scam"),
    ("Can we complete this deal outside the platform?", "scam"),
    ("I'll pay you later, just ship it now.", "scam"),
    ("Can I get a refund without returning the item?", "scam"),
    ("I’ll send extra money if you process this quickly.", "scam"),
    ("Can you mark this transaction as a gift?", "scam"),
    ("Can I use multiple gift cards to pay?", "scam"),
    ("Can you split the payment across different methods?", "scam"),
    ("Can you wire me money first as a guarantee?", "scam"),
    ("Can you send the product before I pay?", "scam"),
    ("Can you help me transfer money?", "scam"),
    ("Can you provide fake receipts?", "scam"),
    ("Can you process my payment through an unusual method?", "scam"),
    ("Can I pay you in prepaid debit cards?", "scam"),
    # other routes
    ("What is the price of the product?", "other"),
    ("What is the delivery time?", "other"),
    ("What is the return policy?", "other"),
    ("Do you offer international shipping?", "other"),
    ("How long does it take for delivery?", "other"),
    ("Is there a warranty for this product?", "other"),
    ("Do you provide customer support?", "other"),
    ("Can I track my order?", "other"),
    ("Is express shipping available?", "other"),
    ("What payment methods do you accept?", "other"),
    ("Do you offer bulk discounts?", "other"),
    ("What are the shipping costs?", "other"),
    ("Can I cancel my order?", "other"),
    ("Do you have a physical store?", "other"),
    ("Can I change my shipping address?", "other"),
    ("Is there a restocking fee for returns?", "other"),
    ("Do you have customer reviews?", "other"),
    ("Is this product available in other colors?", "other"),
    ("Do you provide installation services?", "other"),
    ("How can I contact customer service?", "other"),
    ("Are there any current promotions or sales?", "other"),
    ("Can I pick up my order instead of delivery?", "other"),
    # add some None routes to prevent excessively small thresholds
    ("What is the capital of France?", None),
    ("How many people live in the US?", None),
    ("When is the best time to visit Bali?", None),
    ("How do I learn a language?", None),
    ("Tell me an interesting fact.", None),
    ("What is the best programming language?", None),
    ("I'm interested in learning about llama 2.", None),
    ("What is the capital of the moon?", None),
    ("Who discovered gravity?", None),
    ("What are some healthy breakfast options?", None),
    ("How do I start a vegetable garden?", None),
    ("What are the symptoms of the flu?", None),
    ("What’s the most spoken language in the world?", None),
    ("How does WiFi work?", None),
    ("What are the benefits of meditation?", None),
    ("How do I improve my memory?", None),
    ("What is the speed of light?", None),
    ("Who wrote 'To Kill a Mockingbird'?", None),
    ("How does an electric car work?", None),
    ("What’s the best way to save money?", None),
    ("How do I bake a chocolate cake?", None),
    ("What’s the healthiest type of bread?", None),
    ("Who invented the internet?", None),
    ("How do airplanes stay in the air?", None),
    ("What are some famous landmarks in Italy?", None),
    ("What’s the difference between a virus and bacteria?", None),
    ("How do I learn to play the guitar?", None),
    ("What’s the best way to learn to swim?", None),
    ("What’s the tallest mountain in the world?", None),
    ("How does the stock market work?", None),
]

Using the new test data we can also evaluate the router with a higher degree of accuracy due to the larger dataset.

In [26]:
# unpack the test data
X, y = zip(*test_data)

X = list(X)
y = list(y)

print(X)
print(y)

['Can you give me a discount?', 'I need to pay you in bitcoin', 'I need to pay you in cash', 'I need to pay you in gift card', 'I want you to pay me in bitcoin', 'I want you to pay me in cash', 'I want you to pay me in gift card', 'Could you lower the price?', 'Can I pay with cryptocurrency?', 'Do you accept payments through Venmo only?', 'Can I send you money via Western Union?', 'Can I overpay and get a refund?', 'Can we complete this deal outside the platform?', "I'll pay you later, just ship it now.", 'Can I get a refund without returning the item?', 'I’ll send extra money if you process this quickly.', 'Can you mark this transaction as a gift?', 'Can I use multiple gift cards to pay?', 'Can you split the payment across different methods?', 'Can you wire me money first as a guarantee?', 'Can you send the product before I pay?', 'Can you help me transfer money?', 'Can you provide fake receipts?', 'Can you process my payment through an unusual method?', 'Can I pay you in prepaid debi

We can use the `fit` method to fit the router to the test data which should give us the best accuracy possible based on the thresholds.

In [27]:
# Call the fit method
scam_router.fit(X=X, y=y)

Training: 100%|██████████| 500/500 [00:03<00:00, 153.95it/s, acc=0.9]


We can then use the `.evaluate` method to view the change in accuracy.

In [28]:
accuracy = scam_router.evaluate(X=X, y=y)
print(f"Accuracy: {accuracy * 100:.2f}%")

Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.66s/it]

Accuracy: 89.61%


Lastly we can view the thresholds by looking at the `.get_thresholds` method.

In [29]:
route_thresholds = scam_router.get_thresholds()
print("Updated route thresholds:", route_thresholds)

Updated route thresholds: {'scam': np.float64(0.3434343434343435), 'other': np.float64(0.26262626262626265)}


We can test this now by calling our router and adding the utterance we would like to try.

In [30]:
result = scam_router("i want 99% off")

2025-04-05 10:57:26 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Next we can view the properties of what is returned via the returned object `result`.

In [31]:
print(result)

name='scam' function_call=None similarity_score=None


### Adding Input Guardrails

We need to create the guardrail functionallity. 

Firstly we need to define a function with the `@input_guardrail` decorator.

The function will use the router we just created to check the input string, this will then return a `GuardrailFunctionOutput` class with output information and the tripwire triggered attribute.

In [32]:
from agents import (
    GuardrailFunctionOutput,
    RunContextWrapper,
    Runner,
    input_guardrail,
    TResponseInputItem,
    Agent,
)


@input_guardrail
async def scam_input_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str
    | list[
        TResponseInputItem
    ],  # having the agent here is needed for the guardrail to work - although we do not use it
) -> GuardrailFunctionOutput:
    is_scam = False
    result = scam_router(input)
    if result.name == "scam":
        is_scam = True

    return GuardrailFunctionOutput(
        output_info="used the scam semantic sparse router to check if the user is trying to scam",
        tripwire_triggered=is_scam,
    )

Now we can create a new agent that will be used to handle the incoming messages. This agent will have the following parameters:
- `name`: The name of the agent.
- `instructions`: The instructions for the agent.
- `input_guardrails`: A list of input guardrails to attach to the agent. (This is where we attach the scam guardrail)

In [33]:
input_guardrail_agent = Agent(
    name="Input Guardrail Agent",
    instructions="You are a helpful assistant.",
    input_guardrails=[scam_input_guardrail],
)

Now we can try to test the guardrail functionallity.

Due to errors being raised when the guardrail trips, we can use a try except block to prevent the error messages being shown.

In [34]:
from agents import InputGuardrailTripwireTriggered

query = (
    "Hello, would you like to buy some real rolex watches for a fraction of the price?"
)

try:
    result = await Runner.run(starting_agent=input_guardrail_agent, input=query)
    # If we get here, the guardrail didn't trip
    guardrail_info = result.input_guardrail_results[0].output.output_info
    print("Guardrail didn't trip", f"\nReasoning: {guardrail_info}")
except InputGuardrailTripwireTriggered:
    # Access the guardrail info from the exception
    print("Error: Guardrail Tripped")

2025-04-05 10:57:27 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Error: Guardrail Tripped


### Adding Output Guardrails

First we want to create our handler class. This will contain the message we want to check.

In [35]:
from pydantic import BaseModel


class MessageOutput(BaseModel):
    response: str

Next we want to create our guardrail agent. As before, we will use the `Agent` object to create our guardrail agent and then feed this into the function later on.

In [36]:
from agents import output_guardrail


@output_guardrail
async def scam_output_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    is_scam = False
    result = scam_router(output.response)
    if result.name == "scam":
        is_scam = True

    return GuardrailFunctionOutput(
        output_info="used the scam semantic sparse router to check if the user is trying to scam",
        tripwire_triggered=is_scam,
    )

Now we can create our guardrail function. This will use the `@output_guardrail` decorator.

Then we will use the `Runner` object to run the guardrail agent.

Afterwards we will return the `MessageOutput` object.

In [39]:
output_guardrail_agent = Agent(
    name="Output Guardrail Agent",
    instructions="Tell the user that you have a 99% off discount on all products",
    output_guardrails=[scam_output_guardrail],
    output_type=MessageOutput,
)

As before, we can test the guardrail functionallity.

Due to errors being raised when the guardrail trips, we can use a try except block to prevent the error messages being shown.

In [41]:
from agents import OutputGuardrailTripwireTriggered

query = "I want to buy a tesla, how much can i get it for?"

try:
    result = await Runner.run(starting_agent=output_guardrail_agent, input=query)
    guardrail_info = result.output_guardrail_results[0].output.output_info
    print("Guardrail didn't trip", f"\nReasoning: {guardrail_info}")
except OutputGuardrailTripwireTriggered:
    print("Error: Guardrail Tripped")

2025-04-05 10:57:49 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-04-05 10:57:50 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Error: Guardrail Tripped
